# Redis

On this page I'm getting to know reddis. It's a inmemory no-sql database.

## Using command line

Just use the `redis-cli` command to go into its shell. There is a `ping` command that answers `PONG` to check if everything is OK.

The following cell shows how this can look like.

In [1]:
%%bash
redis-cli
ping

PONG


## String

Is atomic thing in `redis`. There are such commands to operate on it.

|Command|What it does|
|-------|------------|
| `set` | Sets value for the variable|
| `get` | Get value from the key value|
| `del` | Delet value under key|

In [2]:
%%bash
redis-cli

set hello world
get hello

OK
world


After activating `redis-cli` we add a new key/value combination - it returns `OK` and then try to access the value under the `hello` key - it returns `world`, which is the value corresponding to the `hello` key.

Ok now lets try to delete just created key/value pair from redis.

In [3]:
%%bash
redis-cli

del hello
del hello
get hello

1
0



The first execution of `del` returns `1` because the key `hello` exists, but the second `del` returns `0` because the key `hello` has already been deleted. And as proof that the key has been deleted, we try to access it with `get` and get an empty response.

## Lists

Redis has a `list` data structure. There are some operations you can perform on redis lists.

|Command|What it does|
|-------|------------|
|`rpush`|Pushes the value onto the right end of the list|
|`lrange`|Fetches a range of values from the list|
|`lindex`|Fetches an item at a given position in the list|
|`lpop`|Pops the value from the left end of the list and returns it|

Now let's play around with it.

Here some values are pushed to the `list-key`.

In [4]:
%%bash
redis-cli

rpush list-key my
rpush list-key name
rpush list-key Fedor

1
2
3


By using `list-key` we can get values that were stored in the list.

In [5]:
%%bash
redis-cli
lrange list-key 0 -1

my
name
Fedor


By using `lindex` we can access any element you want.

In [6]:
%%bash
redis-cli

lindex list-key 0
lindex list-key 2
lindex list-key 1

my
Fedor
name


Now we can `rpop` items from the list. Each pop returns the last value of the list and clears it.

In [7]:
%%bash
redis-cli

rpop list-key
rpop list-key
rpop list-key

Fedor
name
my


## Sets

Redis has a `set` data structure - it's a bunch of unique strings. There are some commands to use them with redis sets:

|Command|What it does|
|-------|------------|
|`sadd`|Adds an item to the set|
|`smembers`|Returns entire set of items|
|`sismember`|Checks is an item is in set|
|`srem`|Remove the item from the set|

Next cells represents some experiments with this set of commands.

Now let's add some new values to the set. Here we add `"this value"` and `"new value"`, but `"this value"` has been added twice.

In [8]:
%%bash
redis-cli

sadd set-key "this value"
sadd set-key "new value"
sadd set-key "this value"

1
1
0


It's interesting that outputs looks like it returns `1` if the operation was successful and `0` if not. In our case the last `sadd` tried to add an already existing element.

Now, using `smembers`, we can access the set we have just created.

In [9]:
%%bash 
redis-cli
smembers set-key

this value
new value


Some practice for `sismember` and `srem` in one cell.

In [10]:
%%bash
redis-cli

sismember set-key "new value"
srem set-key "new value"
sismember set-key "new value"

1
1
0


Before removing `sismember` it returns `1`, after removing it returns `0`. The middle `1` is the output of the `srem` command - it signals that everything has been successfully completed.

Don't forget to delete the set you just created, to ensure the same execution for this example and to exclude useless garbage in our reddis instance.

In [11]:
%%bash
redis-cli
del set-key

1


## Hashes

`Hash` is a key/value structure in redis. You can think of it as a small sub-redis.

You can operate with hashes using following commands.

|Command|What it does|
|-------|------------|
|`hset`|Stores the value at the key in the hash|
|`hget`|Fetches value at the given hash key|
|`hgetall`|Fetches the entire hash|
|`hdel`|Removes the key from the hash|

Here is an example of making hash with different animals and the sounds they make.

In [12]:
%%bash
redis-cli

hset hash-key cat meow
hset hash-key dog woh
hset hash-key snake ssss
hset hash-key cat mow
hset hash-key bird chirp
hset hash-key dog bark
hset hash-key snake hiss
hset hash-key cat purr
hset hash-key bird tweet

1
1
1
0
1
0
0
0
0


Each time a new record was added to the has, 0 or 1 was printed. 1 is for cases where a new key was created, but 0 is for cases where such a case already exists.

Now let's try `hgetall`.

In [13]:
%%bash
redis-cli
hgetall hash-key

cat
purr
dog
bark
snake
hiss
bird
tweet


Well, result may be a bit uncomfortable to read but it has structure like:

```
<key1>
<value1>
<key2>
<value2>
...
<keyN>
<valueN>
```

Don't forget to delete the hash you just created, to ensure the same execution for this example and to exclude useless garbage in our reddis instance.

In [14]:
%%bash
redis-cli
del hash-key

1


## Zsets

Is a data structure where the number (score) corresponds to the value. This allows you to select values by the range of their scores.

It supports following commands:

|Command|What it does|
|-------|------------|
|`zadd`|Adds member with get given score to zset|
|`zrange`|Fetches the items in the zset from their position in sorted order|
|`zrangebyscore`|Fetches items in the zset based on range of socres|
|`zrem`|Removes the item from zset, if it exists|

Ok let's have experiments with this structure. Here we have zset that ranges aminal acording to their weights.

In [15]:
%%bash
redis-cli

zadd animal-weights 50 elephant
zadd animal-weights 20 lion
zadd animal-weights 30 giraffe
zadd animal-weights 10 cheetah
zadd animal-weights 50 rhinoceros
zadd animal-weights 30 giraffe
zadd animal-weights 25 lion
zadd animal-weights 20 tiger

0
0
0
0
0
0
0
0


It's output is 1 every time I add a new value, but scores can be repeated as often as you like.

Now let us try to apply `zrange` to the zset under consideration:

In [16]:
%%bash
redis-cli
zrange animal-weights 0 -1

cheetah
tiger
lion
giraffe
elephant
rhinoceros


We have all the values that we tried to add once. In order of increasing scores. **Note** that `rhinoceros` have the same score as `elephant` but it diaplyed later because it was added later.

By adding the `withscores` option to the `zrange` command, you can specify that you want to see the scores as well. The following example shows what you'll get:

In [17]:
%%bash
redis-cli
zrange animal-weights 0 -1 withscores

cheetah
10
tiger
20
lion
25
giraffe
30
elephant
50
rhinoceros
50


But now suppose you want to get all animals that have weight between 20 and 40. So you can use `zrangebyscore` command. The following cell displays such option

In [18]:
%%bash
redis-cli
zrangebyscore animal-weights 20 40

tiger
lion
giraffe


Don't forget to delete the zset you just created, to ensure the same execution for this example and to exclude useless garbage in our reddis instance.

In [19]:
%%bash
redis-cli
del animal-weights

1
